## RTA Monitor

Define the *maximum* number of seconds you would like to monitor the process 

> Note that once all heating steps are complete, no more temperature data can be read. The reading process may terminate before the actual specified number of seconds to read for.

In [4]:
# remeber, this number is in seconds!
# The actual reading time may 
READ_FOR = 15*60

Where should I save plot data?

In [5]:
from pathlib import Path 

data_path = Path("RTA-data\Characterization")

Great. Here are some useful `python` libraries that we will be using. Please help me import them by clicking the run button next to the cell.

In [6]:
import h5py
import numpy as np
import plotly.graph_objs as go
import matplotlib.pyplot as plt 

from SerialReader import SerialReaderPlotter

s = SerialReaderPlotter("COM8", baudrate=19200, timeout=60)

## Live Plot
The next cell creates a plot window where our temperature versus time trace will go. Every time you run the following cell, a new graph will be created.

In [31]:
fig = go.FigureWidget(data=[])
fig.update_layout(
    template="plotly_white",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (C)",
)

FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'T…

## Begin
When you are ready, click run in the following cell to start the monitoring process.

In [30]:

"""
In this cell, I left to you the convenience of allowing to other things 
while the serial reader is reading.
"""
s.start_reading(READ_FOR)
# fig.update_layout(xaxis_range=[0,READ_FOR])
s.start_writing(fig)

# s.writer.join()

(Reader   ) Starting
(Plotter  ) Starting


## Saving Data

Write down any short comments about the process 

In [28]:
COMMENT = "Empty_Test_Run__5mbar_330_445"

In [15]:
from datetime import datetime
date = datetime.now().strftime("%Y_%m_%d-%I-%M_%p")
print("Today: {}".format(date))

Today: 2021_08_31-08-09_PM


In [17]:
for i, scatter in enumerate(fig.data):
    filename = data_path/Path("RTA_Run_{}_".format(i) + COMMENT + "_{}.h5".format(date))
    hf = h5py.File(filename, 'w')
    hf.create_dataset('time_arr', data=np.array(scatter.x[1:], dtype=float)) 
    hf.create_dataset('temperature_arr', data=np.array(scatter.y[1:], dtype=float))
    hf.close()

## Loading Data

In [25]:
previous_runs = {f.name: h5py.File(f, "r") for f in data_path.glob("*.h5")}

In [ ]:
read = go.FigureWidget(data=[])
read.update_layout(
    template="plotly_white",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (C)",
)

for k in previous_runs.keys():
    read.add_scatter(x=[], 
                    y=[], 
                    line=dict(width=1))

    read_dat = read.data[-1]
    
    x = previous_runs[k]["time_arr"][:]
    y = previous_runs[k]["temperature_arr"][:]
    with read.batch_update():
        read_dat.x += tuple(x)
        read_dat.y += tuple(y) 
    previous_runs[k].close()
read